# Climatología nubosidad Sudamérica ISCCPH data

Nadia Testani Febrero 2022
## Objetivos

El objetivo de este código es analizar la climatología de la nubosidad en Sudamérica y en distintas subregiones del continente (Sudeste de SA -SESA-, Noreste Argentino -NEA- y Corrientes Argentina -Corrientes-) a partir de productos satelitales de nubosidad de ISCCPH

## Datos

Los datos con los que se trabajan son el producto mensual de nubosidad total (cldamt) y tipos de nubosidad (cldamt types) de ISCCPH. Los mismos vienen dados como archivos NetCDF.

## Estructura del código

El código se estructura de la siguiente manera:

* En primer lugar se cargan las librerias necesarias para el código y los módulos propios (f_graficos_climatologia_isccph y f_procesamiento_satelite)

* Luego se cargan las rutas a los datos y a los resultados

* Luego se cargan los datos

* Luego se calculan las variables climatológicas: media y desvio y las variables de cambios: tendencia lineal

* Por último se realizan los gráficos sobre las variables calculadas

------------------------------

### Cargo librerias necesarias para el código


In [1]:
import warnings
warnings.filterwarnings("ignore") #para que muestre los warnings una sola vez

import os
import geopandas
from cartopy.io import shapereader
from shapely.geometry.multipolygon import MultiPolygon
import cmocean
import pandas as pd
import xarray as xr
import numpy as np
from shapely.geometry import mapping, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from f_graficos_climatologia_isccph import grafico_campos_climatologia_nubosidad, grafico_campos_climatologia_nubosidad_tendencias, serie_media_espacial, clip_corrientes
from f_procesamiento_satelite import cldamt_tipo, calculo_media_isccp, calculo_desvio_isccp, calculo_tendencia_isccp

### Cargo rutas

In [2]:
cwd = os.getcwd()
print("* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *\n")
print(f"Current working directory: {cwd}")
print("")
print("* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *")

trabajo_en_mi_pc = False
trabajo_en_vegeta = True

if trabajo_en_mi_pc:
    None
    #Obs: los datos no estan cargados en la PC, hay que descargarlos antes de usar el codigo
    #si estoy trabajando desde mi pc parada en directorio ./Doctorado/codigos/codigos2021/nubosidad
    ruta_a_archivos_isccp_mensual = "../../../datos/nubosidad/ISCCP-H_HGM/" #ruta donde estan los archivos con datos del smn
    ruta_a_resultados = "../../../resultados/resultados2021/nubosidad/" #ruta donde se guardan los resultados del analisis de nubosidad
     
elif trabajo_en_vegeta:
    #si estoy trabajando desde el servidor vegeta
    ruta_a_archivos_isccp_diario = "/pikachu/datos2/isccp-h/HGG" #ruta donde estan los archivos con datos de isccp-h
    ruta_a_archivos_isccp_mensual = "/pikachu/datos2/isccp-h/HGM" #ruta donde estan los archivos con datos de isccp-h
    ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/" #ruta donde se guardan los resultados del analisis de nubosidad


* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

Current working directory: /home/nadia.testani/Doctorado/codigos/codigos2021/nubosidad

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *


### Cargo datos ISCCP-H mensuales

In [3]:
isccp_mensual = xr.open_dataset(f"{ruta_a_archivos_isccp_mensual}/media_mensual_cldamt_cldamttypes.nc")[["cldamt","cldamt_types"]]
isccp_mensual = isccp_mensual.loc[dict(lon=slice(270, 331),lat=slice(-61,16))] #recorto region de interes
isccp_mensual = isccp_mensual.sel(time=slice("1983-12-15", "2016-11-15"))

print("* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *\n")
print(f"Los datos mensuales de nubosidad ISCCPH ya estan cargados y recortados en la region de interes, en las fechas de interes")
print("")
print("* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *")


* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

Los datos mensuales de nubosidad ISCCPH ya estan cargados y recortados en la region de interes, en las fechas de interes

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *


### Acomodo los datos en variables de interes

* isccp_mensual_cldamt: nubosidad mensual
* isccp_mensual_bajas: nubosidad mensual baja - suma de cldamt_types [0:6]
* isccp_mensual_medias: nubosidad mensual media - suma de cldamt_types [6:12]
* isccp_mensual_altas: nubosidad mensual alta - suma de cldamt_types [12:19]
* isccp_mensual_alturas: diccionario donde se acomodan isccp_mensual_bajas, isccp_mensual_medias y isccp_mensual_altas

In [4]:
isccp_mensual_cldamt = isccp_mensual["cldamt"]
isccp_mensual_bajas = cldamt_tipo(isccp_mensual, "BAJAS")
isccp_mensual_medias = cldamt_tipo(isccp_mensual, "MEDIAS")
isccp_mensual_altas = cldamt_tipo(isccp_mensual, "ALTAS")
isccp_mensual_alturas = {"Low clouds": isccp_mensual_bajas,
                         "Middle clouds": isccp_mensual_medias,
                         "High clouds": isccp_mensual_altas}

**Las variables anteriores se extienden para Corrientes (se regrillan los datos y se recorta por el shape de corrientes), se llaman a nuevas variables así:**

* isccp_mensual_cldamt_corrientes: nubosidad mensual
* isccp_mensual_bajas_corrientes: nubosidad mensual baja - suma de cldamt_types [0:6]
* isccp_mensual_medias_corrientes: nubosidad mensual media - suma de cldamt_types [6:12]
* isccp_mensual_altas_corrientes: nubosidad mensual alta - suma de cldamt_types [12:19]
* isccp_mensual_alturas_corrientes: diccionario donde se acomodan isccp_mensual_bajas, isccp_mensual_medias y isccp_mensual_altas

In [5]:
isccp_mensual_cldamt_corrientes =  clip_corrientes(isccp_mensual_cldamt)
isccp_mensual_bajas_corrientes =  clip_corrientes(isccp_mensual_bajas)
isccp_mensual_medias_corrientes =  clip_corrientes(isccp_mensual_medias)
isccp_mensual_altas_corrientes =  clip_corrientes(isccp_mensual_altas)

isccp_mensual_alturas_corrientes = {"Low clouds": isccp_mensual_bajas_corrientes,
                                    "Middle clouds": isccp_mensual_medias_corrientes,
                                    "High clouds": isccp_mensual_altas_corrientes}

### Calculo variables de climatologia

Se calculan las siguientes variables: 

* media anual
* media anual para las tres alturas de nubes

* media estacional
* media estacional para las tres alturas de nubes (para cada estacion las tres nubes en un diccionario)
 
* media mensual 
* media mensual para las tres alturas de nubes (para cada tipo de nube los 12 meses en un diccionario)
 
* desvio anual 
* desvio anual para las tres alturas de nubes
 

In [6]:
media = calculo_media_isccp(xrdata = isccp_mensual_cldamt, mensual = False, estacional = False)
media_alturas = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = False),
                "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = False),
                "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = False)}

media_estacional = calculo_media_isccp(xrdata = isccp_mensual_cldamt, mensual = False, estacional = True)
media_alturas_DJF = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["DJF"],
                     "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["DJF"],
                     "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["DJF"]}
media_alturas_MAM = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["MAM"],
                      "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["MAM"],
                      "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["MAM"]}
media_alturas_JJA = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["JJA"],
                      "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["JJA"],
                      "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["JJA"]}
media_alturas_SON = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["SON"],
                     "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["SON"],
                     "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["SON"]}

media_mensual = calculo_media_isccp(xrdata = isccp_mensual_cldamt, mensual = True, estacional = False)
media_mensual_bajas = calculo_media_isccp(xrdata = isccp_mensual_bajas, mensual = True, estacional = False)
media_mensual_medias = calculo_media_isccp(xrdata = isccp_mensual_medias, mensual = True, estacional = False)
media_mensual_altas = calculo_media_isccp(xrdata = isccp_mensual_altas, mensual = True, estacional = False)

desvio = calculo_desvio_isccp(xrdata = isccp_mensual_cldamt, mensual = False, estacional = False)
desvio_alturas = {"Low clouds": calculo_desvio_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = False),
                "Middle clouds": calculo_desvio_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = False),
                "High clouds": calculo_desvio_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = False)}


**Se extienden los mismos calculos con las varaibles de Corrientes**

In [7]:
media_corrientes = calculo_media_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = False, estacional = False)
media_alturas_corrientes = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = False),
                            "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = False),
                            "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = False)}


media_estacional_corrientes = calculo_media_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = False, estacional = True)
media_alturas_DJF_corrientes = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["DJF"],
                                "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["DJF"],
                                "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["DJF"]}

media_alturas_MAM_corrientes = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["MAM"],
                                "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["MAM"],
                                "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["MAM"]}

media_alturas_JJA_corrientes = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["JJA"],
                                "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["JJA"],
                                "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["JJA"]}

media_alturas_SON_corrientes = {"Low clouds": calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["SON"],
                                "Middle clouds": calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["SON"],
                                "High clouds": calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["SON"]}

media_mensual_corrientes = calculo_media_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = True, estacional = False)
media_mensual_bajas_corrientes = calculo_media_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = True, estacional = False)
media_mensual_medias_corrientes = calculo_media_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = True, estacional = False)
media_mensual_altas_corrientes = calculo_media_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = True, estacional = False)


desvio_corrientes = calculo_desvio_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = False, estacional = False)
desvio_alturas_corrientes = {"Low clouds": calculo_desvio_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = False),
                            "Middle clouds": calculo_desvio_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = False),
                            "High clouds": calculo_desvio_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = False)}




### Calculo variables de cambios

Se calculan las siguientes variables: 

* tendencia 
* tendencia para las tres alturas de nubes (cada altura un elemento de diccionario)

* tendencia estacional
* tendencia estacional para las tres alturas de nubes (para cada estacion los tres tipos de nubes como diccionario)

*Obs:* Se calcula la tendencia y si es significativa con un 95% de confianza con un test de Rank Kendall


In [8]:
tendencia = calculo_tendencia_isccp(xrdata = isccp_mensual_cldamt, mensual = False, estacional = False) 
#tendencia_alturas = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = False),
  #                   "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = False),
  #                   "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = False)}

tendencia_estacional = calculo_tendencia_isccp(xrdata = isccp_mensual_cldamt, mensual = False, estacional = True)
#tendencia_alturas_DJF = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["DJF"],
#                         "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["DJF"],
#                         "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["DJF"]}
#tendencia_alturas_MAM = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["MAM"],
#                         "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["MAM"],
 #                        "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["MAM"]}
#tendencia_alturas_JJA = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["JJA"],
#                         "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["JJA"],
 #                        "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["JJA"]}
#tendencia_alturas_SON = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas, mensual = False, estacional = True)["SON"],
 #                        "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias, mensual = False, estacional = True)["SON"],
  #                       "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas, mensual = False, estacional = True)["SON"]}

**Se extienden los mismos calculos con las varaibles de Corrientes**

In [ ]:
tendencia_corrientes = calculo_tendencia_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = False, estacional = False) 
tendencia_alturas_corrientes = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = False),
                                "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = False),
                                "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = False)}

tendencia_estacional_corrientes = calculo_tendencia_isccp(xrdata = isccp_mensual_cldamt_corrientes, mensual = False, estacional = True)
tendencia_alturas_DJF_corrientes = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["DJF"],
                                    "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["DJF"],
                                    "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["DJF"]}
tendencia_alturas_MAM_corrientes = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["MAM"],
                                    "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["MAM"],
                                    "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["MAM"]}
tendencia_alturas_JJA_corrientes = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["JJA"],
                                    "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["JJA"],
                                    "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["JJA"]}
tendencia_alturas_SON_corrientes = {"Low clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_bajas_corrientes, mensual = False, estacional = True)["SON"],
                                    "Middle clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_medias_corrientes, mensual = False, estacional = True)["SON"],
                                    "High clouds": calculo_tendencia_isccp(xrdata = isccp_mensual_altas_corrientes, mensual = False, estacional = True)["SON"]}

### Grafico las variables calculadas

Se hacen los siguientes gráficos (en dos dimensiones: una para poster y otra para PPT):

#### Climatologia:

* Media anual de nubosidad
* Media anual de nubosidad para los tres tipos de nubes (en un mismo grafico)

* Media estacional de nubosidad (las cuatro estaciones en un mismo grafico)
* Media estacional de nubosidad baja (las cuatro estaciones en un mismo grafico)
* Media estacional de nubosidad media (las cuatro estaciones en un mismo grafico) 
* Media estacional de nubosidad alta (las cuatro estaciones en un mismo grafico) 

* Media mensual de nubosidad (los 12 meses en un mismo grafico)
* Media mensual de nubosidad baja (los 12 meses en un mismo grafico)
* Media mensual de nubosidad media (los 12 meses en un mismo grafico)
* Media mensual de nubosidad alta (los 12 meses en un mismo grafico)

* Desvio anual de nubosidad
* Desvio anual de nubosidad para los tres tipos de nubes (en un mismo grafico)

In [ ]:
def ejecuto_graficos_climatologia(region, poster):

    #Media Anual
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Annual mean", tres_tipos_de_nubes = False)

    ##Media Anual alturas
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = media_alturas, nombre_variable = "cldamt", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Annual mean", tres_tipos_de_nubes = True)
    #Media Estacional
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_estacional, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = True, mensual = False, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean", tres_tipos_de_nubes = False)
    
    #Media estacional alturas DJF
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = media_alturas_DJF, nombre_variable = "cldamt_types", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Seasonal mean (DJF)", tres_tipos_de_nubes = True)

    #Media estacional alturas MAM
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = media_alturas_MAM, nombre_variable = "cldamt_types", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Seasonal mean (MAM)", tres_tipos_de_nubes = True)

    #Media estacional alturas JJA
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = media_alturas_JJA, nombre_variable = "cldamt_types", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Seasonal mean (JJA)", tres_tipos_de_nubes = True)
    #Media estacional alturas SON
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = media_alturas_SON, nombre_variable = "cldamt_types", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Seasonal mean (SON)", tres_tipos_de_nubes = True)
    
    ##Media mensual
    #grafico_campos_climatologia_nubosidad(xrdata = media_mensual, nombre_variable = "cldamt", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = True, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Monthly mean", tres_tipos_de_nubes = False)
    ##Media mensual nubes bajas 
    #grafico_campos_climatologia_nubosidad(xrdata = media_mensual_bajas, nombre_variable = "cldamt", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = True, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Monthly mean (low clouds)", tres_tipos_de_nubes = False)
    
    ##Media mensual nubes medias
    #grafico_campos_climatologia_nubosidad(xrdata = media_mensual_medias, nombre_variable = "cldamt", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = True, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Monthly mean (middle clouds)", tres_tipos_de_nubes = False)
    ##Media mensual nubes altas
    #grafico_campos_climatologia_nubosidad(xrdata = media_mensual_altas, nombre_variable = "cldamt", 
    #                                      unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
    #                                      delta_valor = 5, estacional = False, mensual = True, 
    #                                      region = region, paleta_color = "rain", grid = False, poster = poster, 
    #                                      estadistico = "Monthly mean (high clouds)", tres_tipos_de_nubes = False)
  
    ##Desvio anual
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = desvio, nombre_variable = "cldamt",unidades_nombre = "%", 
    #                                      valor_minimo = 0, valor_maximo =40, 
    #                                      delta_valor = 2, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "matter", grid = False, poster = poster,
    #                                      estadistico = "Annual standard deviation", tres_tipos_de_nubes = False)

    ##Desvio anual alturas
    #plt.close()
    #grafico_campos_climatologia_nubosidad(xrdata = desvio_alturas, nombre_variable = "cldamt_types",unidades_nombre = "%", 
    #                                      valor_minimo = 0, valor_maximo =40, 
    #                                      delta_valor = 2, estacional = False, mensual = False, 
    #                                      region = region, paleta_color = "matter", grid = False, poster = poster,
    #                                      estadistico = "Annual standard deviation", tres_tipos_de_nubes = True)



In [ ]:
#ejecuto graficos climatologia

def ejecuto_graficos_climatologia_corrientes(poster):

    #Media Anual
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Annual mean", tres_tipos_de_nubes = False)

    #Media Anual alturas
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Annual mean", tres_tipos_de_nubes = True)
    #Media Estacional
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_estacional_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = True, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean", tres_tipos_de_nubes = False)
    
    #Media estacional alturas DJF
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_DJF_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (DJF)", tres_tipos_de_nubes = True)

    #Media estacional alturas MAM
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_MAM_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (MAM)", tres_tipos_de_nubes = True)

    #Media estacional alturas JJA
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_JJA_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (JJA)", tres_tipos_de_nubes = True)
    #Media estacional alturas SON
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_SON_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (SON)", tres_tipos_de_nubes = True)
    
    #Media mensual
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = True, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean", tres_tipos_de_nubes = False)
    #Media mensual nubes bajas 
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_bajas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = True, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (low clouds)", tres_tipos_de_nubes = False)
    
    #Media mensual nubes medias
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_medias_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = True, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (middle clouds)", tres_tipos_de_nubes = False)
    #Media mensual nubes altas
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_altas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 0, valor_maximo =101, 
                                          delta_valor = 5, estacional = False, mensual = True, 
                                          region = region, paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (high clouds)", tres_tipos_de_nubes = False)
  
    
    
    #Desvio anual
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = desvio_corrientes, nombre_variable = "cldamt",unidades_nombre = "%", 
                                          valor_minimo = 0, valor_maximo =40, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = region, paleta_color = "matter", grid = False, poster = poster,
                                          estadistico = "Annual standard deviation", tres_tipos_de_nubes = False)

    #Desvio anual alturas
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = desvio_alturas_corrientes, nombre_variable = "cldamt_types",unidades_nombre = "%", 
                                          valor_minimo = 0, valor_maximo =40, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "matter", grid = False, poster = poster,
                                          estadistico = "Annual standard deviation", tres_tipos_de_nubes = True)



In [ ]:
#ejecuto graficos climatologia

def ejecuto_graficos_climatologia_corrientes_2(poster):
    #cambia las escalas se guarda con un espacio al final del nombre del estadistico
    #Media Anual
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 45, valor_maximo =65, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Annual mean ", tres_tipos_de_nubes = False)

    #Media Anual alturas
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Annual mean ", tres_tipos_de_nubes = True)
    #Media Estacional
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_estacional_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 45, valor_maximo =65, 
                                          delta_valor = 2, estacional = True, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean ", tres_tipos_de_nubes = False)
    
    #Media estacional alturas DJF
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_DJF_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (DJF) ", tres_tipos_de_nubes = True)

    #Media estacional alturas MAM
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_MAM_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (MAM) ", tres_tipos_de_nubes = True)

    #Media estacional alturas JJA
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_JJA_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (JJA) ", tres_tipos_de_nubes = True)
    #Media estacional alturas SON
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = media_alturas_SON_corrientes, nombre_variable = "cldamt_types", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Seasonal mean (SON) ", tres_tipos_de_nubes = True)
    
    #Media mensual
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 45, valor_maximo =65, 
                                          delta_valor = 2, estacional = False, mensual = True, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean ", tres_tipos_de_nubes = False)
    #Media mensual nubes bajas 
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_bajas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = True, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (low clouds) ", tres_tipos_de_nubes = False)
    
    #Media mensual nubes medias
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_medias_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = True, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (middle clouds) ", tres_tipos_de_nubes = False)
    #Media mensual nubes altas
    grafico_campos_climatologia_nubosidad(xrdata = media_mensual_altas_corrientes, nombre_variable = "cldamt", 
                                          unidades_nombre = "%", valor_minimo = 5, valor_maximo =35, 
                                          delta_valor = 2, estacional = False, mensual = True, 
                                          region = "Corrientes", paleta_color = "rain", grid = False, poster = poster, 
                                          estadistico = "Monthly mean (high clouds) ", tres_tipos_de_nubes = False)
  
    
    
    #Desvio anual
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = desvio_corrientes, nombre_variable = "cldamt",unidades_nombre = "%", 
                                          valor_minimo = 0, valor_maximo =20, 
                                          delta_valor = 1, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "matter", grid = False, poster = poster,
                                          estadistico = "Annual standard deviation ", tres_tipos_de_nubes = False)

    #Desvio anual alturas
    plt.close()
    grafico_campos_climatologia_nubosidad(xrdata = desvio_alturas_corrientes, nombre_variable = "cldamt_types",unidades_nombre = "%", 
                                          valor_minimo = 0, valor_maximo =20, 
                                          delta_valor = 1, estacional = False, mensual = False, 
                                          region = "Corrientes", paleta_color = "matter", grid = False, poster = poster,
                                          estadistico = "Annual standard deviation ", tres_tipos_de_nubes = True)



In [ ]:
#ejecuto graficos cambios

def ejecuto_graficos_cambios(region, poster):

    #Anual
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = region, 
                                                     grid = False, poster = poster, estadistico = "Annual tendency", 
                                                     tres_tipos_de_nubes = False)

    #Anual alturas
    #plt.close()
    #grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas, 
    #                                                 unidades_nombre = "%/dec", valor_minimo = -8 , 
    #                                                 valor_maximo = 9, delta_valor = 1, estacional = False, 
    #                                                 mensual = False, region = region, 
    #                                                 grid = False, poster = poster, estadistico = "Annual tendency", 
    #                                                 tres_tipos_de_nubes = True)
    #Estacional
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_estacional, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = True, 
                                                     mensual = False, region = region, 
                                                     grid = False, poster = poster, estadistico = "Seasonal tendency", 
                                                     tres_tipos_de_nubes = False)
    #Estacional alturas DJF
    #plt.close()
    #grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_DJF, 
    #                                                 unidades_nombre = "%/dec", valor_minimo = -8 , 
    #                                                 valor_maximo = 9, delta_valor = 1, estacional = False, 
    #                                                 mensual = False, region = region, 
    #                                                 grid = False, poster = poster, 
    #                                                 estadistico = "Seasonal tendency (DJF)", 
    #                                                 tres_tipos_de_nubes = True)

    #Estacional alturas MAM
    #plt.close()
    #grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_SON, 
    #                                                 unidades_nombre = "%/dec", valor_minimo = -8 , 
    #                                                 valor_maximo = 9, delta_valor = 1, estacional = False, 
    #                                                 mensual = False, region = region, 
    #                                                 grid = False, poster = poster, 
    #                                                 estadistico = "Seasonal tendency (MAM)", 
    #                                                 tres_tipos_de_nubes = True)

    #Estacional alturas JJA
    #plt.close()
    #grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_JJA, 
    #                                                 unidades_nombre = "%/dec", valor_minimo = -8 , 
    #                                                 valor_maximo = 9, delta_valor = 1, estacional = False, 
    #                                                 mensual = False, region = region, 
    #                                                 grid = False, poster = poster, 
    #                                                 estadistico = "Seasonal tendency (JJA)", 
    #                                                 tres_tipos_de_nubes = True)
    #Estacional alturas SON
    #plt.close()
    #grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_SON, 
    #                                                 unidades_nombre = "%/dec", valor_minimo = -8 , 
    #                                                 valor_maximo = 9, delta_valor = 1, estacional = False, 
    #                                                 mensual = False, region = region, 
    #                                                 grid = False, poster = poster, 
    #                                                 estadistico = "Seasonal tendency (SON)", 
    #                                                 tres_tipos_de_nubes = True)
    #series temporales
    #serie_media_espacial(isccp_mensual_cldamt, isccp_mensual_alturas, estacional = False, region = region, poster = poster)
    


In [ ]:
#ejecuto graficos cambios corrientes

def ejecuto_graficos_cambios_corrientes(poster):

    #Anual
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, estadistico = "Annual tendency", 
                                                     tres_tipos_de_nubes = False)

    #Anual alturas
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, estadistico = "Annual tendency", 
                                                     tres_tipos_de_nubes = True)
    #Estacional
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_estacional_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = True, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, estadistico = "Seasonal tendency", 
                                                     tres_tipos_de_nubes = False)
    #Estacional alturas DJF
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_DJF_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, 
                                                     estadistico = "Seasonal tendency (DJF)", 
                                                     tres_tipos_de_nubes = True)

    #Estacional alturas MAM
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_SON_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, 
                                                     estadistico = "Seasonal tendency (MAM)", 
                                                     tres_tipos_de_nubes = True)

    #Estacional alturas JJA
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_JJA_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, 
                                                     estadistico = "Seasonal tendency (JJA)", 
                                                     tres_tipos_de_nubes = True)
    #Estacional alturas SON
    plt.close()
    grafico_campos_climatologia_nubosidad_tendencias(xrdata = tendencia_alturas_SON_corrientes, 
                                                     unidades_nombre = "%/dec", valor_minimo = -8 , 
                                                     valor_maximo = 9, delta_valor = 1, estacional = False, 
                                                     mensual = False, region = "Corrientes", 
                                                     grid = False, poster = poster, 
                                                     estadistico = "Seasonal tendency (SON)", 
                                                     tres_tipos_de_nubes = True)
    #series temporales 
    serie_media_espacial(isccp_mensual_cldamt_corrientes, isccp_mensual_alturas_corrientes, estacional = False, region = "Corrientes", poster = poster)
    

In [ ]:
#ejecuto graficos
print("####### Manejo de directorios #######")
print("\n")
print(f"Se para en el directorio llamado: {ruta_a_resultados}")
os.chdir(ruta_a_resultados) #se para en ruta general de resultados de nubosidad
print("\n")
for region in ["SA", "NEA", "SESA"]:
    if region not in os.listdir():
        print(f"genera directorio llamado: {region} en {ruta_a_resultados}")
        os.mkdir(region)
    else:
        print(f"se para en el directorio llamado: {region} en {ruta_a_resultados}")
    os.chdir(f"./{region}") 
    print("\n")

    if "poster" not in os.listdir():
        print(f"genera directorio llamado: poster en {ruta_a_resultados}{region}")
        os.mkdir("poster")
    else:
        print(f"se para en el directorio llamado: poster en {ruta_a_resultados}{region}")
    os.chdir("./poster") 
    print("\n")

    ejecuto_graficos_climatologia(region = region, poster = True)
    ejecuto_graficos_cambios(region = region, poster = True)

    print("\n")
    #vuelvo a region
    os.chdir("./..")
    cwd = os.getcwd()
    print(f"Vuelvo a {region}. Ahora el directorio de trabajo es: {0}".format(cwd))
    print("\n")

    if "no_poster" not in os.listdir():
        print(f"genera directorio llamado: no_poster en {ruta_a_resultados}{region}")
        os.mkdir("no_poster")
    else:
        print(f"se para en el directorio llamado: no_poster en {ruta_a_resultados}{region}")
    os.chdir("./no_poster") #se para en poster
    print("\n")

    ejecuto_graficos_climatologia(region = region, poster = False)
    ejecuto_graficos_cambios(region = region, poster = False)

    print("\n")
    #vuelvo a resultados
    os.chdir("./../..")
    cwd = os.getcwd()
    print(f"Vuelvo a {ruta_a_resultados}. Ahora el directorio de trabajo es: {cwd}")
    print("\n")

In [ ]:
#ejecuto graficos corrientes
print("####### Manejo de directorios #######")
print("\n")
print(f"Se para en el directorio llamado: {ruta_a_resultados}")
os.chdir(ruta_a_resultados) #se para en ruta general de resultados de nubosidad
print("\n")
region = "Corrientes"
if region not in os.listdir():
    print(f"genera directorio llamado: {region} en {ruta_a_resultados}")
    os.mkdir(region)
else:
    print(f"se para en el directorio llamado: {region} en {ruta_a_resultados}")
os.chdir(f"./{region}") 
print("\n")

if "poster" not in os.listdir():
    print(f"genera directorio llamado: poster en {ruta_a_resultados}{region}")
    os.mkdir("poster")
else:
    print(f"se para en el directorio llamado: poster en {ruta_a_resultados}{region}")
os.chdir("./poster") 
print("\n")

#ejecuto_graficos_climatologia_corrientes(poster = True)
ejecuto_graficos_climatologia_corrientes_2(poster = True)
#ejecuto_graficos_cambios_corrientes(poster = True)

print("\n")
#vuelvo a region
os.chdir("./..")
cwd = os.getcwd()
print(f"Vuelvo a {region}. Ahora el directorio de trabajo es: {0}".format(cwd))
print("\n")

if "no_poster" not in os.listdir():
    print(f"genera directorio llamado: no_poster en {ruta_a_resultados}{region}")
    os.mkdir("no_poster")
else:
    print(f"se para en el directorio llamado: no_poster en {ruta_a_resultados}{region}")
os.chdir("./no_poster") #se para en poster
print("\n")

ejecuto_graficos_climatologia_corrientes(poster = False)
ejecuto_graficos_climatologia_corrientes_2(poster = False)
ejecuto_graficos_cambios_corrientes(poster = False)

print("\n")
#vuelvo a resultados
os.chdir("./../..")
cwd = os.getcwd()
print(f"Vuelvo a {ruta_a_resultados}. Ahora el directorio de trabajo es: {cwd}")
print("\n")

In [ ]:
import geopandas
from cartopy.io import shapereader
from shapely.geometry.multipolygon import MultiPolygon
import cmocean
import pandas as pd
import xarray as xr
import numpy as np
from shapely.geometry import mapping, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pymannkendall as mk
from f_mis_shapes import df_paises, df_provincias



def grafico_campos_climatologia_nubosidad(xrdata, nombre_variable,unidades_nombre, valor_minimo, valor_maximo, delta_valor, estacional = False, mensual = False, region = "SA", paleta_color = "rain", grid = False, poster = True, estadistico = "annual mean", tres_tipos_de_nubes = False):
    plt.close()
    if region == "SA":
        lat_min = -60
        lat_max = 15
        lon_min = -90
        lon_max = -30
        xticks_min = -85
        xticks_max = -30
        yticks_min = -55
        yticks_max = 15

    elif region == "NEA":
        lat_min = -32
        lat_max = -22
        lon_min = -64
        lon_max = -53
        xticks_min = -63
        xticks_max = -53
        yticks_min = -31
        yticks_max = -22

    elif region == "SESA":
        lat_min = -39
        lat_max = -16
        lon_min = -64
        lon_max = -31
        xticks_min = -60
        xticks_max = -31
        yticks_min = -35
        yticks_max = -18

    elif region == "Corrientes": 
        lat_min = -31
        lat_max = -26
        lon_min = -60
        lon_max = -55
        xticks_min = -59
        xticks_max = -55
        yticks_min = -30
        yticks_max = -26
        
    if tres_tipos_de_nubes == False:
        if estacional == False and mensual == False:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 12
                else:
                    espacio_entre_lat_lon = 8
            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 2
                else:
                    espacio_entre_lat_lon = 2
            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 4 
                else:
                    espacio_entre_lat_lon = 4
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 1
                else:
                    espacio_entre_lat_lon = 1
                    
            # selecciono region
            data_region = xrdata.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
            #data = xrdata.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
            
            # ploteo 
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[5, 3.5], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[7.5, 7.5], dpi=200) 

            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 3.5], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[8.5, 5.5], dpi=200)  
            
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6.5, 4], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[9, 5.5], dpi=200) 
                    
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[5.5, 4], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[8.5, 5.5], dpi=200) 

            ax = fig1.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))

            if (paleta_color == "rain"):
                data_region.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',
                                     transform=ccrs.PlateCarree(),cbar_kwargs={'label': "%"},cmap=cmocean.cm.rain)

            elif (paleta_color == "matter"):
                data_region.plot.contourf(ax=ax, levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',
                                     transform=ccrs.PlateCarree(),cbar_kwargs={'label': "%"},cmap=cmocean.cm.matter)

            # seteo el fondo gris para que los nans aparezcan en este color
            if region != "Corrientes": 
                ax.set_facecolor('tab:gray')
                
            ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

            ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

            ax.coastlines(color='0.3')

            #ax.set_xlim(np.min(data_region.lon.values) - 360, np.max(data_region.lon.values)-360)
            ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
            plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
            ax.set_xlabel("Lon")
            ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])

            #ax.set_ylim(np.min(data_region.lat.values), np.max(data_region.lat.values))
            plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
            ax.set_ylabel("Lat")

            if (grid == True):
                plt.grid(linestyle="--", alpha=0.3)

            plt.title(f"CA: {estadistico} \nDec1983-Nov2016")
            # plt.tight_layout()
            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}.png")
            plt.show()


        elif estacional == True and mensual == False:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 16
                else:
                    espacio_entre_lat_lon = 12

            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 2
                else:
                    espacio_entre_lat_lon = 2

            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 8 
                else:
                    espacio_entre_lat_lon = 6 
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 2 
                else:
                    espacio_entre_lat_lon = 2 
                    
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[4, 4], dpi=1000)  # vertical sudamerica POSTER
                else:
                    fig1 = plt.figure(figsize=[7.5, 7], dpi=200)  # vertical sudamerica VER

            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[5.5, 4], dpi=1000)  # nea POSTER
                else:
                    fig1 = plt.figure(figsize=[7.5, 5.5], dpi=200)  # nea
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6.5, 3.8], dpi=1000)  # ver
                else:
                    fig1 = plt.figure(figsize=[7.5, 4.3], dpi=200)  # ver
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[4.8, 4], dpi=1000)  # ver
                else:
                    fig1 = plt.figure(figsize=[6.5, 5.5], dpi=200)  # ver

            posiciones = [221, 222, 223, 224]
            estaciones = ["DJF", "MAM", "JJA", "SON"]
            estaciones_nombre = ["DEF", "MAM", "JJA", "SON"]

            for i in range(0,4):
                xrdata_estacion = xrdata[estaciones[i]] #selecciono estacion del diccionario xrdata
                data_estacion = xrdata_estacion.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]

                ax = fig1.add_subplot(posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))

                if (paleta_color == "rain"):
                    im = data_estacion.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo+1, delta_valor), transform=ccrs.PlateCarree(),cmap=cmocean.cm.rain, extend='neither', cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []})

                elif (paleta_color == "matter"):
                    im = data_estacion.plot.contourf(ax=ax, levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',transform=ccrs.PlateCarree(),cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []},cmap=cmocean.cm.matter)

                # seteo el fondo gris para que los nans aparezcan en este color
                if region != "Corrientes": 
                    ax.set_facecolor('tab:gray')
                
                ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

                ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

                ax.coastlines(color='0.3')

                #ax.set_xlim(np.min(data_estacion.lon.values) - 360, np.max(data_estacion.lon.values)-360)
                plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                if i == 2 or i ==3 : #solo para los de abajo
                    ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                    ax.set_xlabel("Lon")
                else:
                    ax.set_xticklabels([])
                    ax.set_xlabel("")

                #ax.set_ylim(np.min(data_estacion.lat.values) - 360, np.max(data_estacion.lat.values)-360)
                plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                if i == 0 or i == 2 : #solo para los de la izquierda
                    ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                    ax.set_ylabel("Lat")
                else:
                    ax.set_yticklabels([])
                    ax.set_ylabel("")

                if (grid == True):
                    plt.grid(linestyle="--", alpha=0.3)
                ax.set_title(estaciones_nombre[i]) #CAMBIAR LUEGO DE POSTER POR estaciones

            plt.subplots_adjust(wspace=-0.3)
            cbar_ax = fig1.add_axes([0.85, 0.13, 0.05, 0.75])
            fig1.colorbar(im, cax=cbar_ax, label = unidades_nombre )

            plt.suptitle(f"CA: {estadistico} Dec1983-Nov2016")


            #plt.colorbar()
            #plt.tight_layout()
            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_estaciones_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}_estaciones.png")
            plt.show()
            
        elif estacional == False and mensual == True:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 20
                else:
                    espacio_entre_lat_lon = 16

            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 4
                else:
                    espacio_entre_lat_lon = 3

            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 8 
                else:
                    espacio_entre_lat_lon = 7 
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 2 
                else:
                    espacio_entre_lat_lon = 2 
                    
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[4.2, 6.2], dpi=200)  # vertical sudamerica POSTER [4,6]
                else:
                    fig1 = plt.figure(figsize=[4.9, 7.2], dpi=200)  # vertical sudamerica VER

            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[4.4, 5.3], dpi=200)  # nea POSTER
                else:
                    fig1 = plt.figure(figsize=[6.3, 7], dpi=200)  # nea
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[5.1, 5.2], dpi=200)  # ver [4.9, 4.4]
                else:
                    fig1 = plt.figure(figsize=[7.1, 7], dpi=200)  # ver [8.1, 7]
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[4.4, 5.3], dpi=200)  # ver
                else:
                    fig1 = plt.figure(figsize=[6.3, 7], dpi=200)  # ver

            posiciones = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2), (3,0), (3,1), (3,2)]
            meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
            meses_nombre = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Ag", "Sep", "Oct", "Nov", "Dec"]

            for i in range(0,12):
                xrdata_mes = xrdata[meses[i]] #selecciono mes del diccionario xrdata
                data_mes = xrdata_mes.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
                ax = plt.subplot2grid((4,3), posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))
                
                #ax = fig1.add_subplot(posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))

                if (paleta_color == "rain"):
                    im = data_mes.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo+1, delta_valor), transform=ccrs.PlateCarree(),cmap=cmocean.cm.rain, extend='neither', cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []})

                elif (paleta_color == "matter"):
                    im = data_mes.plot.contourf(ax=ax, levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',transform=ccrs.PlateCarree(),cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []},cmap=cmocean.cm.matter)

                # seteo el fondo gris para que los nans aparezcan en este color
                if region != "Corrientes": 
                    ax.set_facecolor('tab:gray')
                
                ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

                ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

                ax.coastlines(color='0.3')

                #ax.set_xlim(np.min(data_estacion.lon.values) - 360, np.max(data_estacion.lon.values)-360)
                plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                if i == 9 or i == 10 or i == 11 : #solo para los de abajo
                    ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                    ax.set_xlabel("Lon")
                else:
                    ax.set_xticklabels([])
                    ax.set_xlabel("")

                #ax.set_ylim(np.min(data_estacion.lat.values) - 360, np.max(data_estacion.lat.values)-360)
                plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                if i == 0 or i == 3 or i == 6 or i == 9 : #solo para los de la izquierda
                    ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                    ax.set_ylabel("Lat")
                else:
                    ax.set_yticklabels([])
                    ax.set_ylabel("")

                if (grid == True):
                    plt.grid(linestyle="--", alpha=0.3)
                ax.set_title(meses_nombre[i])

            plt.subplots_adjust(wspace=-0.3, hspace=0.4)
            cbar_ax = fig1.add_axes([0.85, 0.13, 0.05, 0.75])
            fig1.colorbar(im, cax=cbar_ax, label = unidades_nombre )

            plt.suptitle(f"CA: {estadistico} Dec1983-Nov2016", y=0.95)


            #plt.colorbar()
            #plt.tight_layout()
            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_meses_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}_meses.png")
            plt.show()      
                
    elif tres_tipos_de_nubes == True:
        if estacional == False and mensual == False:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 12
                else:
                    espacio_entre_lat_lon = 12
            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 3
                else:
                    espacio_entre_lat_lon = 3
            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 8 
                else:
                    espacio_entre_lat_lon = 8
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 2
                else:
                    espacio_entre_lat_lon = 2
            
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.7], dpi=200)  # vertical sudamerica POSTER
                else:
                    fig1 = plt.figure(figsize=[8, 3.5], dpi=200)  # vertical sudamerica
            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.2], dpi=200)  # 
                else:
                    fig1 = plt.figure(figsize=[8.5, 2.8], dpi=200)  # 
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 1.9], dpi=200)  # 
                else:
                    fig1 = plt.figure(figsize=[8.5, 2.4], dpi=200)  # 
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.5], dpi=200)  # 
                else:
                    fig1 = plt.figure(figsize=[8, 3], dpi=200)  # 

            posiciones = [131, 132, 133]
            alturas = ["Low clouds", "Middle clouds", "High clouds"]

            for i in range(0,3):
                xrdata_altura = xrdata[alturas[i]] #selecciono altura de la nube del diccionario xrdata
                data_altura = xrdata_altura.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]

                ax = fig1.add_subplot(posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))

                if (paleta_color == "rain"):
                    im = data_altura.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo+1, delta_valor), transform=ccrs.PlateCarree(),cmap=cmocean.cm.rain, extend='neither', cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []})

                elif (paleta_color == "matter"):
                    im = data_altura.plot.contourf(ax=ax, levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',transform=ccrs.PlateCarree(),cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []},cmap=cmocean.cm.matter)

                # seteo el fondo gris para que los nans aparezcan en este color
                if region != "Corrientes": 
                    ax.set_facecolor('tab:gray')
                
                ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

                ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

                ax.coastlines(color='0.3')

                ax.set_xlim(np.min(data_altura.lon.values) - 360, np.max(data_altura.lon.values)-360)
                plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])

                ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                ax.set_xlabel("Lon")


                ax.set_ylim(np.min(data_altura.lat.values), np.max(data_altura.lat.values))
                plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                if i == 0 : #solo para los de la izquierda
                    ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                    ax.set_ylabel("Lat")
                else:
                    ax.set_yticklabels([])
                    ax.set_ylabel("")

                if (grid == True):
                    plt.grid(linestyle="--", alpha=0.3)
                ax.set_title(alturas[i])

            plt.subplots_adjust(wspace=-0.1)
            cbar_ax = fig1.add_axes([0.88, 0.2, 0.05, 0.6])
            fig1.colorbar(im, cax=cbar_ax, label = unidades_nombre )

            plt.suptitle(f"CA: {estadistico} Dec1983-Nov2016")


            #plt.colorbar()
            #plt.tight_layout()
            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_alturas_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}_alturas.png")
            plt.show()
                        


In [ ]:
def grafico_campos_climatologia_nubosidad_tendencias(xrdata, unidades_nombre, valor_minimo, valor_maximo, delta_valor, estacional = False, mensual = False, region = "SA", grid = False, poster = True, estadistico = "Annual tendency", tres_tipos_de_nubes = False):
    
    plt.close()
    if region == "SA":
        lat_min = -60
        lat_max = 15
        lon_min = -90
        lon_max = -30
        xticks_min = -85
        xticks_max = -30
        yticks_min = -55
        yticks_max = 15

    elif region == "NEA":
        lat_min = -32
        lat_max = -22
        lon_min = -64
        lon_max = -53
        xticks_min = -63
        xticks_max = -53
        yticks_min = -31
        yticks_max = -22

    elif region == "SESA":
        lat_min = -39
        lat_max = -16
        lon_min = -64
        lon_max = -31
        xticks_min = -60
        xticks_max = -31
        yticks_min = -35
        yticks_max = -18

    elif region == "Corrientes": 
        lat_min = -31
        lat_max = -26
        lon_min = -60
        lon_max = -55
        xticks_min = -59
        xticks_max = -55
        yticks_min = -30
        yticks_max = -26
        
    if tres_tipos_de_nubes == False:
        if estacional == False and mensual == False:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 12
                else:
                    espacio_entre_lat_lon = 8
            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 2
                else:
                    espacio_entre_lat_lon = 2
            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 4 
                else:
                    espacio_entre_lat_lon = 4
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 1 
                else:
                    espacio_entre_lat_lon = 1 

            # recorto region
            variable_data = xrdata[0].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
            variable_significancia = xrdata[1].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
            # me quedo solo con los que son significativos para marcar esas regiones con puntitos
            variable_significancia = variable_significancia.where(variable_significancia.values == 1)

            # ploteo 
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[5, 3.5], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[7.5, 7.5], dpi=200) 

            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 3.5], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[8.5, 5.5], dpi=200)  
            
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6.5, 4], dpi=1000)  #ver
                else:
                    fig1 = plt.figure(figsize=[9, 5.5], dpi=200) #ver
            
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[5.5, 4], dpi=1000)  # [4,4]
                else:
                    fig1 = plt.figure(figsize=[8.5, 5.5], dpi=200) #ver
            

            ax = fig1.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))
            
            
            variable_data.plot.contourf(ax=ax, levels=np.arange(valor_minimo, valor_maximo, delta_valor),
                                            extend='both',transform=ccrs.PlateCarree(),
                                            cbar_kwargs={'label': unidades_nombre},cmap=cmocean.cm.balance)
        
                
            # grafico puntos en los lugares donde es significativo
            variable_significancia.plot.contourf(ax=ax, levels=[0, 1], hatches=['.'], alpha=0, add_colorbar=False)

            # seteo el fondo gris para que los nans aparezcan en este color
            if region != "Corrientes": 
                ax.set_facecolor('tab:gray')

            ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',
                              edgecolor='0.5', linewidth=0.7, alpha=0.9)

            ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',
                              edgecolor='0.4', alpha=0.9)

            ax.coastlines(color='0.3')

            ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
            plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
            ax.set_xlabel("Lon")

            ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
            plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
            ax.set_ylabel("Lat")

            if (grid == True):
                plt.grid(linestyle="--", alpha=0.3)

            plt.title(f"CA: {estadistico} \nDec1983-Nov2016")

            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_poster.png")
                plt.show()
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}.png")
                plt.show()

        elif estacional == True and mensual == False: #xrdata es un diccionario donde cada elemento es un xarray 
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 16
                else:
                    espacio_entre_lat_lon = 12

            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 2
                else:
                    espacio_entre_lat_lon = 2

            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 8 
                else:
                    espacio_entre_lat_lon = 6 
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 2 
                else:
                    espacio_entre_lat_lon = 2 
            
            
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[4, 4], dpi=1000) 
                else:
                    fig1 = plt.figure(figsize=[7.5, 7.5], dpi=200)  

            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[5.5, 4], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[7.5, 5.5], dpi=200)  
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6.5, 3.8], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[7.5, 4.3], dpi=200)  
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[4.8, 4], dpi=1000)  
                else:
                    fig1 = plt.figure(figsize=[6.5, 5.5], dpi=200)  


            posiciones = [221, 222, 223, 224]
            estaciones = ["DJF", "MAM", "JJA", "SON"]
            estaciones_nombres = ["DEF", "MAM", "JJA", "SON"]

            for i in range(0,4):
                xrdata_estacion = xrdata[estaciones[i]] #selecciono estacion del diccionario xrdata
                variable_data_estacion = xrdata_estacion[0].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
                variable_significancia_estacion = xrdata_estacion[1].loc[dict(lon=slice(360+lon_min,360+lon_max), lat=slice(lat_min, lat_max))]
                # me quedo solo con los que son significativos para marcar esas regiones con puntitos
                variable_significancia_estacion = variable_significancia_estacion.where(variable_significancia_estacion.values == 1)

                ax = fig1.add_subplot(posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))

                im = variable_data_estacion.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='both',transform=ccrs.PlateCarree(),cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []},cmap=cmocean.cm.balance)

                # grafico puntos en los lugares donde es significativo
                variable_significancia_estacion.plot.contourf(ax=ax, levels=[0, 1], hatches=["."], alpha=0, add_colorbar=False) 

                # seteo el fondo gris para que los nans aparezcan en este color
                if region != "Corrientes": 
                    ax.set_facecolor('tab:gray')
                

                ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

                ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

                ax.coastlines(color='0.3')
                
                ax.set_xlim(np.min(variable_data_estacion.lon.values) - 360, np.max(variable_data_estacion.lon.values)-360)
                plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                if i == 2 or i ==3 : #solo para los de abajo
                    ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                    ax.set_xlabel("Lon")
                else:
                    ax.set_xticklabels([])
                    ax.set_xlabel("")
                
                ax.set_ylim(np.min(variable_data_estacion.lat.values), np.max(variable_data_estacion.lat.values))
                plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                if i == 0 or i == 2 : #solo para los de la izquierda
                    ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                    ax.set_ylabel("Lat")
                else:
                    ax.set_yticklabels([])
                    ax.set_ylabel("")

                if (grid == True):
                    plt.grid(linestyle="--", alpha=0.3)
                
                ax.set_title(estaciones_nombres[i])

            plt.subplots_adjust(wspace=-0.3)
            cbar_ax = fig1.add_axes([0.85, 0.13, 0.05, 0.75])
            fig1.colorbar(im, cax=cbar_ax, label = unidades_nombre )

            plt.suptitle(f"CA: {estadistico} Dec1983-Nov2016")

            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_estaciones_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}_estaciones.png")
            plt.show()
            
    elif tres_tipos_de_nubes == True:     
        
        if estacional == False and mensual == False:
            if region == "SA":
                if poster:
                    espacio_entre_lat_lon = 12
                else:
                    espacio_entre_lat_lon = 12
            elif region == "NEA":
                if poster:
                    espacio_entre_lat_lon = 3
                else:
                    espacio_entre_lat_lon = 3
            elif region == "SESA":
                if poster:
                    espacio_entre_lat_lon = 8 #ver
                else:
                    espacio_entre_lat_lon = 8
            elif region == "Corrientes":
                if poster:
                    espacio_entre_lat_lon = 2 #ver
                else:
                    espacio_entre_lat_lon = 2
            
            if region == "SA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.7], dpi=200)  
                else:
                    fig1 = plt.figure(figsize=[8, 3.5], dpi=200)  
            elif region == "NEA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.2], dpi=200)   
                else:
                    fig1 = plt.figure(figsize=[8.5, 2.8], dpi=200)  
            elif region == "SESA":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 1.9], dpi=200)  
                else:
                    fig1 = plt.figure(figsize=[8.5, 2.4], dpi=200)  
            elif region == "Corrientes":
                if poster == True:
                    fig1 = plt.figure(figsize=[6, 2.5], dpi=200)  
                else:
                    fig1 = plt.figure(figsize=[8, 3], dpi=200)  

            posiciones = [131, 132, 133]
            alturas = ["Low clouds", "Middle clouds", "High clouds"]
            
            for i in range(0,3):
                xrdata_altura = xrdata[alturas[i]] #selecciono altura de la nube del diccionario xrdata
                variable_data = xrdata_altura[0].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
                variable_significancia = xrdata_altura[1].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
                # me quedo solo con los que son significativos para marcar esas regiones con puntitos
                variable_significancia = variable_significancia.where(variable_significancia.values == 1)

                ax = fig1.add_subplot(posiciones[i], projection=ccrs.PlateCarree(central_longitude=0))
                
                im = variable_data.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='both',transform=ccrs.PlateCarree(),cbar_kwargs = {'shrink':0.0001, "label":" ", "ticks": []},cmap=cmocean.cm.balance)

                # grafico puntos en los lugares donde es significativo
                variable_significancia.plot.contourf(ax=ax, levels=[0, 1], hatches=["."], alpha=0, add_colorbar=False) 

                # seteo el fondo gris para que los nans aparezcan en este color
                if region != "Corrientes": 
                    ax.set_facecolor('tab:gray')

                ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)

                ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)

                ax.coastlines(color='0.3')
                
                ax.set_xlim(np.min(variable_data.lon.values) - 360, np.max(variable_data.lon.values)-360)
                plt.xticks(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                ax.set_xticklabels(np.arange(xticks_min, xticks_max)[::espacio_entre_lat_lon])
                ax.set_xlabel("Lon")


                ax.set_ylim(np.min(variable_data.lat.values), np.max(variable_data.lat.values))
                plt.yticks(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                if i == 0 : #solo para los de la izquierda
                    ax.set_yticklabels(np.arange(yticks_min, yticks_max)[::espacio_entre_lat_lon])
                    ax.set_ylabel("Lat")
                else:
                    ax.set_yticklabels([])
                    ax.set_ylabel("")

                if (grid == True):
                    plt.grid(linestyle="--", alpha=0.3)
                ax.set_title(alturas[i])
                
            plt.subplots_adjust(wspace=-0.1)
            cbar_ax = fig1.add_axes([0.88, 0.2, 0.05, 0.6])
            fig1.colorbar(im, cax=cbar_ax, label = unidades_nombre )
            
            plt.suptitle(f"CA: {estadistico} Dec1983-Nov2016")

            if poster:
                plt.savefig(f"cldamt_{estadistico}_{region}_alturas_poster.png")
            else:
                plt.savefig(f"cldamt_{estadistico}_{region}_alturas.png")
            plt.show()


In [ ]:
#ejecuto graficos SA POSTER CONGREMET
print("####### Manejo de directorios #######")
print("\n")
print(f"Se para en el directorio llamado: {ruta_a_resultados}")
os.chdir(ruta_a_resultados) #se para en ruta general de resultados de nubosidad
print("\n")
region = "SA"
if region not in os.listdir():
    print(f"genera directorio llamado: {region} en {ruta_a_resultados}")
    os.mkdir(region)
else:
    print(f"se para en el directorio llamado: {region} en {ruta_a_resultados}")
os.chdir(f"./{region}") 
print("\n")

if "poster" not in os.listdir():
    print(f"genera directorio llamado: poster en {ruta_a_resultados}{region}")
    os.mkdir("poster")
else:
    print(f"se para en el directorio llamado: poster en {ruta_a_resultados}{region}")
os.chdir("./poster") 
print("\n")

ejecuto_graficos_climatologia(region = region, poster = True)
ejecuto_graficos_cambios(region = region, poster = True)

print("\n")
#vuelvo a region
os.chdir("./..")
cwd = os.getcwd()
print(f"Vuelvo a {region}. Ahora el directorio de trabajo es: {0}".format(cwd))
print("\n")


**Hago graficos paper NEA:**

In [ ]:
xrdata_total = isccp_mensual_cldamt 
xrdata_alturas = isccp_mensual_alturas
  
lat_min = -32
lat_max = -25.5
lon_min = -61
lon_max = -54

#recorto region
serie_cldamt_total = xrdata_total.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
serie_cldamt_bajas = xrdata_alturas["Low clouds"].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
serie_cldamt_medias = xrdata_alturas["Middle clouds"].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
serie_cldamt_altas = xrdata_alturas["High clouds"].loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]

#armo series medias para esa region 
serie_media_cldamt_total = serie_cldamt_total.mean(dim = ["lat","lon"], skipna=True)
serie_media_cldamt_bajas = serie_cldamt_bajas.mean(dim = ["lat","lon"], skipna=True)
serie_media_cldamt_medias = serie_cldamt_medias.mean(dim = ["lat","lon"], skipna=True)
serie_media_cldamt_altas = serie_cldamt_altas.mean(dim = ["lat","lon"], skipna=True)

#calculo media climatologica mensual
media_regional_mensual_total = [np.mean(serie_media_cldamt_total[pd.DatetimeIndex(serie_media_cldamt_total.time).month == mes].values) for mes in [1,2,3,4,5,6,7,8,9,10,11,12]]
media_regional_mensual_bajas = [np.mean(serie_media_cldamt_bajas[pd.DatetimeIndex(serie_media_cldamt_bajas.time).month == mes].values) for mes in [1,2,3,4,5,6,7,8,9,10,11,12]]
media_regional_mensual_medias = [np.mean(serie_media_cldamt_medias[pd.DatetimeIndex(serie_media_cldamt_medias.time).month == mes].values) for mes in [1,2,3,4,5,6,7,8,9,10,11,12]]
media_regional_mensual_altas = [np.mean(serie_media_cldamt_altas[pd.DatetimeIndex(serie_media_cldamt_altas.time).month == mes].values) for mes in [1,2,3,4,5,6,7,8,9,10,11,12]]

In [ ]:
width = 0.98      #ancho de las barras
meses = ['J', 'F', 'M', 'A', 'M', "J", "J", "A", "S", "O","N", "D"] #inicial meses
plt.figure(num=1, figsize=(8, 5), dpi=600) #genero figura
ax = plt.gca()
ax.bar([1,2,3,4,5,6,7,8,9,10,11,12], media_regional_mensual_total, width,color = "k")
ax.set_ylabel('Cloud Amount %')
plt.ylim(40, 65)
plt.xlim(0.5, 12.5)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
ax.set_xticklabels(meses)
plt.xlabel("Month")
plt.savefig(f"climatologia_media_nubosidad_isccp_paper_nea.png")
plt.show()

In [ ]:
import geopandas
from cartopy.io import shapereader
from shapely.geometry.multipolygon import MultiPolygon
import cmocean
import pandas as pd
import xarray as xr
import numpy as np
from shapely.geometry import mapping, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pymannkendall as mk
from f_mis_shapes import df_paises, df_provincias

xrdata = media
valor_minimo = 31.25
valor_maximo = 68.75 
delta_valor = 1

plt.close()

lat_min = -31.5
lat_max = -25.5
lon_min = -61.5
lon_max = -54.5
xticks_min = -61.5
xticks_max = -54.5
yticks_min = -31.5
yticks_max = -25.5

# selecciono region
data_region = xrdata.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]
#data = xrdata.loc[dict(lon=slice(360+lon_min, 360+lon_max),lat=slice(lat_min, lat_max))]

# ploteo 
fig1 = plt.figure(figsize=[8.5, 5.5], dpi=600)  
            
ax = fig1.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))

data_region.plot.contourf(ax=ax,levels=np.arange(valor_minimo, valor_maximo, delta_valor),extend='neither',
                                     transform=ccrs.PlateCarree(),cbar_kwargs={'label': "%"},cmap=cmocean.cm.rain)


ax.set_facecolor('tab:gray')
                
ax.add_geometries(df_provincias(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.5', linewidth=0.7, alpha=0.9)
ax.add_geometries(df_paises(), crs=ccrs.PlateCarree(), facecolor='none',edgecolor='0.4', alpha=0.9)
ax.coastlines(color='0.3')

ax.set_xticklabels(np.arange(xticks_min, xticks_max))
plt.xticks(np.arange(xticks_min, xticks_max))
ax.set_xlabel("Lon")
ax.set_yticklabels(np.arange(yticks_min, yticks_max))
plt.yticks(np.arange(yticks_min, yticks_max))
ax.set_ylabel("Lat")

plt.grid(linestyle="--", alpha=0.3)
plt.show()
